In [1]:
#imports y globals.
import shared.constants  as  const
import shared.supportedFiles as sf
#Recuperar los ficheros con los modelos modificados de la parte 1 para evitar volver a realizar todo el proceso cada vez.
#y el original de entrenamiento
TestFile= const.CSV_FILES[const.TEST_KEY_PART_2]
TrainFile= const.CSV_FILES[const.TRAIN_KEY_PART_2]


In [2]:
#añadir la columna is_female del train original al subconjunto generado en el apartado 1 y el test_id que lo usaremos en la prediccion
TrainFileInicial = const.CSV_FILES[const.TRAIN_KEY]

colum_is_female= TrainFileInicial.csvFile['is_female']
colum_train_id= TrainFileInicial.csvFile['train_id']

TrainFile.csvFile['is_female']= colum_is_female
TrainFile.csvFile['train_id']= colum_train_id
#renombrar columna test_id a train_id para homoheneizar con el del modelo de train.
TrainFile.csvFile.rename(columns= {"train_id" : "test_id"}, inplace=True)
TrainFile.csvFile.head()

KeyError: 'test_id'

In [3]:
#Modelos predictivos y calidad de los mismos con curva ROC
#      Se usan varios algoritmos de prediccion para determinar cuál es el mejor:
#        predictColumn es la columna a ser predecida. El modelo entrenado se asume que es el contenido en la instancia del helper. Self.
#        identifierColumn: normalmente es una columna que hace las veces de indexador identificando de forma unica un registro.
#        testModel: El modelo de test contra el que se realiza la predicción. (Pandas Dataframe)
#        
#        Antes de realizar la predicción el algoritmo limpia el conjunto de datos de test de aquellas columnas que no existan en el modelo entrenado.
#
#        El modo Silent activa la salida de texto informativo
#        El modo ROC_Curve genera la curva ROC para cada algoritmo ayudando de forma visual a la determinación del mejor.
#        Retorna el mejor algoritmo en base a la mejor precisión para cada uno de los algorimos empleados en la estimación que son los siguientes:
#          - Regresión logística
#          - Random Forest
#          - Perceptrón
#          - Árboles de decisión
#          - Bayesiano (Naybe)
#         - K-NeigthBors (Vecindad de 5)
# -----------------------------------------------------------------------------------------------------------------------------------------------
 
TrainFile.getBestPredictionAlgorithm('is_female', 'test_id', TestFile.csvFile, Silent=False, ROC_Curve=True)

KeyError: "['is_female'] not found in axis"